# Postdam PM2.5 Linear Regression Forcasting 

* With Ridge And Lasso Regression Models
* Between 2013 and 2023, data collected by DEBB021 was used.
* To increase the accuracy of PM2.5 data estimation, NO2, O3, SO2, PM10 pollutant gas data accepted by the EEA was added.


# TODO:
* decompose data maybe? But my data does not have trends etc.???
* Set index as Start_timestamp
* Feature Engineering ?!?!?! PCA ?!?!?! 
* Separate train/val/test data 60-20-20, explain why?
* Create Model
* Train model
* Evaluate with metrics
* Test
* Predict
* Plot Visualise

* Add Ringe and Lasso Alg
* Evaluate them
* HyperParamater Tune them



**Decomposition:** ?!?!?!? ()
Decompose the time series into trend, seasonal, and residual components. You could model the trend (which may be non-linear) separately and use linear regression to model the residuals or seasonal variations.

In [1]:
# imports
import os
import pandas as pd
os.chdir('..')
import matplotlib.pyplot as plt
import numpy as np, pandas as pd
from src.linear_regression import decompose_time_series

## Data Exploration

* Load Data


In [2]:
df_clean = pd.read_csv('data/CLEAN_MERGED_DE_DEBB021.csv')
df_clean.head()

,Start_Timestamp,End_Timestamp,PM2.5-Pollutant,PM2.5-Value,PM2.5-Unit,PM2.5-Validity,PM2.5-Verification,PM10-Pollutant,PM10-Value,PM10-Unit,...,O3-Pollutant,O3-Value,O3-Unit,O3-Validity,O3-Verification,SO2-Pollutant,SO2-Value,SO2-Unit,SO2-Validity,SO2-Verification
0,1356998400,1357002000,6001,71.04,ug.m-3,1,1,5,88.96,ug.m-3,...,7,43.17,ug.m-3,1,1,1,12.18,ug.m-3,1,1
1,1357002000,1357005600,6001,20.52,ug.m-3,1,1,5,25.17,ug.m-3,...,7,57.15,ug.m-3,1,1,1,4.65,ug.m-3,1,1
2,1357005600,1357009200,6001,9.56,ug.m-3,1,1,5,11.97,ug.m-3,...,7,63.31,ug.m-3,1,1,1,1.33,ug.m-3,1,1
3,1357009200,1357012800,6001,9.45,ug.m-3,1,1,5,11.73,ug.m-3,...,7,63.18,ug.m-3,1,1,1,1.33,ug.m-3,1,1
4,1357012800,1357016400,6001,13.02,ug.m-3,1,1,5,15.88,ug.m-3,...,7,61.70,ug.m-3,1,1,1,1.33,ug.m-3,1,1


In [4]:
df_clean.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Start_Timestamp,87648.0,1.514763e+09,9.108699e+07,1.356998e+09,1.435881e+09,1.514763e+09,1.593645e+09,1.672528e+09
End_Timestamp,87648.0,1.514767e+09,9.108699e+07,1.357002e+09,1.435884e+09,1.514767e+09,1.593649e+09,1.672531e+09
PM2.5-Pollutant,87648.0,6.001000e+03,0.000000e+00,6.001000e+03,6.001000e+03,6.001000e+03,6.001000e+03,6.001000e+03
PM2.5-Value,87648.0,1.303341e+01,1.018415e+01,7.700000e-01,6.000000e+00,1.003000e+01,1.700000e+01,1.940000e+02
PM2.5-Validity,87648.0,1.005545e+00,1.495142e-01,-1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
PM2.5-Verification,87648.0,1.000011e+00,3.377762e-03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
PM10-Pollutant,87648.0,5.000000e+00,0.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00
PM10-Value,87648.0,1.786535e+01,1.249411e+01,8.000000e-01,9.300000e+00,1.491000e+01,2.320000e+01,7.053000e+02
PM10-Validity,87648.0,9.974557e-01,1.234824e-01,-1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
PM10-Verification,87648.0,1.000011e+00,3.377762e-03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00
